In [ ]:
import numpy as np
import os
import sys
import torch
import torch.backends.cudnn as cudnn
import torch.cuda as tcuda
import torch.nn as nn
import yaml

from torch.nn.functional import softmax
from utils import get_data_loader

from sklearn.metrics import (
    auc,
    accuracy_score,
    average_precision_score,
    precision_recall_curve,
    roc_curve
)

from suepvision.smodels import (
    LeNet5,
    get_resnet18,
    get_resnet50
)

In [ ]:
MODEL_NAME = 'ResNet50-LabFrame-Classifier'
config = yaml.safe_load(open("configs/resnet50-labframe-classifier.yml"))
dataset = config['dataset']
evaluation_pref = config['evaluation_pref']

In [ ]:
model = eval(config['architecture'])()
model.load_state_dict(torch.load("models/{}.pth".format(MODEL_NAME)))
model = model.to(0)

In [ ]:
test_loader = get_data_loader(
    dataset['test'][0],
    evaluation_pref['batch_size'],
    evaluation_pref['workers'],
    dataset['in_dim'],
    0,
    boosted=dataset['boosted'],
    shuffle=False
)

In [ ]:
results = torch.tensor([])
model.eval()
with torch.no_grad():
    for images, targets, _, _ in test_loader:
        targets = tcuda.LongTensor(targets, device=0)
        outputs = model(images)
        batch_results = torch.cat((targets.reshape(-1, 1), outputs), 1)
        results = torch.cat((results, batch_results.cpu()), 0)

In [ ]:
CV = 10

y_true = results[:, 0]
y_pred = softmax(results[:, 1:], 1)
y_pred = y_pred[:, 1].detach().cpu().numpy()

samples_per_cv = len(y_true) // CV

aucs, aps, acc = [], [], []
for i in range(CV):
    y_true_cv = y_true[i*samples_per_cv:(i+1)*samples_per_cv]
    y_pred_cv = y_pred[i*samples_per_cv:(i+1)*samples_per_cv]

    fpr, tpr, _ = roc_curve(y_true_cv, y_pred_cv)

    aucs.append(auc(fpr, tpr))
    acc.append(accuracy_score(y_true_cv, y_pred_cv > .5))
    aps.append(average_precision_score(y_true_cv, y_pred_cv))

print(u'Accuracy: {0:.3f} \u00B1 {1:.3f}'.format(np.mean(acc), np.std(acc)))
print(u'ROC AUC: {0:.3f} \u00B1 {1:.3f}'.format(np.mean(aucs), np.std(aucs)))
print(u'PR AP: {0:.3f} \u00B1 {1:.3f}'.format(np.mean(aps), np.std((aps))))